In [4]:
# ALL OF THIS CODE IS INCOMPLETE AND DOES NOT REFLECT THE SOLUTION TO YOUR PROJECT IN PART OR FULLY. PLEASE EDIT AS YOU SEE FIT.
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk

from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt


from nltk import pos_tag
#nltk.download('punkt')#, if you need "tokenizers/punkt/english.pickle", choose it
# nltk.download('averaged_perceptron_tagger')
wnl = nltk.wordnet.WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

from sklearn.feature_extraction import text
stop_words_skt = text.ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
#nltk.download('stopwords' )
stop_words_en = stopwords.words('english')

print("# of stop_words_skt:\t\t %s" % len(stop_words_skt))
print("# of stop_words_en:\t\t %s" % len(stop_words_en))
from string import punctuation
print("# of punctuation:\t\t %s" % len(punctuation))
combined_stopwords = set.union(set(stop_words_en),set(punctuation),set(stop_words_skt))

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(list_word): 
    # Text input is string, returns array of lowercased strings(words).
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]

def stem_rmv_punc(doc): # this should have been at the sentence-level because the pos-tag performs best at sentence-level
    return (word for word in lemmatize_sent(analyzer(doc)) if word not in combined_stopwords and not word.isdigit())


logistic = LogisticRegression(max_iter=10000, tol=0.1)
count_vectorizer = CountVectorizer(min_df=3, stop_words='english', analyzer=stem_rmv_punc)
tfidf = TfidfTransformer()
scaler = StandardScaler(with_mean=False) # not absolutely required but does help
pca = TruncatedSVD()

from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'comp.sys.mac.hardware']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

# I'm doubling up here to make the computations comparable to our dataset's dimensions
full_training = twenty_train.data + twenty_train.data
full_target = list(twenty_train.target) + list(twenty_train.target)
print(len(full_training))


X_digits, y_digits = full_training, full_target

import time
full_time = time.time()
for i in range(36): # let's do some math: cleaning vs. not cleaning (2), stem/lemm/none (3), LSI/NMF (2), Classifiers (3)
    start_time = time.time()
    #cachedir = mkdtemp() # without this, you get results where each iteration takes about 5 minutes on my computer.
    
    pipe = Pipeline(steps=[ 
                       ("vect", count_vectorizer),
                       ('tfidf', tfidf), # please do not split training and testing inside your gridsearch.
                       ("scaler", scaler), 
                       ("pca", pca), 
                       ("logistic", logistic)]) # memory=cachedir), "clf",get the model list)
    
    # Parameters of pipelines can be set using ‘__’ separated parameter names:
    param_grid = {
        "pca__n_components": [10, 100, 1000], # an upper bound
        "logistic__C": [0.01], # your specific model
        "vect__min_df":[2, 6], # min df = 3,5 - these are currently junk values!
    }
    
    search = GridSearchCV(pipe, param_grid, n_jobs=-1, cv=5) # n_jobs is saying how many processes do you want to start concurrently
    # If you are using Google Colab, you need to make sure that you have enough memory. If you don't,
    # your runtime is significantly affected, because the virtual machine will start adding everything to swap memory and 
    # this **really** slows down the processing.
    
    search.fit(X_digits, y_digits)
    print("Best parameter (CV score=%0.3f):" % search.best_score_) 
    # built-in functions exist for this
    # RandomizedSearchCV that you can use as an alternate, but you may not score full points on the search.
    
    print(search.best_params_)
    print("--- %s seconds ---" % (time.time() - start_time))
    rmtree(cachedir)

print("--- %s seconds ---" % (time.time() - full_time))

# of stop_words_skt:		 318
# of stop_words_en:		 179
# of punctuation:		 32
2324
Best parameter (CV score=1.000):
{'logistic__C': 0.01, 'pca__n_components': 1000, 'vect__min_df': 6}
--- 344.99953413009644 seconds ---


KeyboardInterrupt: 